# Accessing info with `cirq-superstaq`
This tutorial will cover the information you can access on your account and related jobs and targets using `cirq-superstaq`.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Infleqtion/client-superstaq/blob/main/docs/source/get_started/access_info/access_info_css.ipynb) [![Launch Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Infleqtion/client-superstaq/HEAD?labpath=docs/source/get_started/access_info/access_info_css.ipynb)

## Imports and API Token

As usual, we'll begin with importing requirements and setting up access to Superstaq. This tutorial uses `cirq-superstaq`, our Superstaq client for Cirq. You can install it and relevant dependencies by running `pip install cirq-superstaq`.

In [1]:
# Requirements to use cirq-superstaq
try:
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq_superstaq as css

import cirq

Now, we instantiate a service provider in `cirq-superstaq` with `Service()`. Supply the Superstaq API token by providing the token as an argument of `css.Service()` or setting it as an environment variable (see [this guide](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
service = css.Service()

## Account Information

The `service` class gives you a means to retrieve information regarding your Superstaq account. Currently, you can use `service` to retrieve your Superstaq balance.

In [3]:
service.get_balance()

'876.39 credits'

If are interested in increasing your balance or have more information on your user role, please reach out to us at superstaq@infleqtion.com or join our [Slack workspace](https://join.slack.com/t/superstaq/shared_invite/zt-1wr6eok5j-fMwB7dPEWGG~5S474xGhxw).

## Backend Information
In addition to account information, the ``Service`` object also gives you a list of all the devices and simulators to which you have access, as well as additional information about those targets.

* `get_targets()`: Retrieves a list of available targets. This method also accepts the following boolean keyword arguments to filter the targets returned: `simulator`, `supports_submit`, `supports_submit_qubo`, `supports_compile`, `available`, `retired`, and `accessible`.
* `get_my_targets()`: Retrieves a filtered list of targets that are accessible to the user based on their user role permissions. This method is equivalent to `get_targets(accessible=True)`.
* `target_info("<target_name>")`: Retrieve information on your selected backend, such as number of qubits, native gate set, where `<target_name>` is the name of the desired backend

In [4]:
service.get_targets()

[Target(target='aqt_keysight_qpu', supports_submit=False, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='aqt_zurich_qpu', supports_submit=False, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='aws_dm1_simulator', supports_submit=True, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='aws_sv1_simulator', supports_submit=True, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='aws_tn1_simulator', supports_submit=True, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='cq_sqale_simulator', supports_submit=True, supports_submit_qubo=False, supports_compile=True, available=True, retired=False, accessible=True),
 Target(target='eeroq_wonderlake_qpu', supports_submi

In [5]:
service.target_info("ibmq_kingston_qpu")  # retrieving information about IBM's Kingston device

{'num_qubits': 156,
 'target': 'ibmq_kingston_qpu',
 'coupling_map': [[0, 1],
  [1, 0],
  [1, 2],
  [2, 1],
  [2, 3],
  [3, 2],
  [3, 4],
  [3, 16],
  [4, 3],
  [4, 5],
  [5, 4],
  [5, 6],
  [6, 5],
  [6, 7],
  [7, 6],
  [7, 8],
  [7, 17],
  [8, 7],
  [8, 9],
  [9, 8],
  [9, 10],
  [10, 9],
  [10, 11],
  [11, 10],
  [11, 12],
  [11, 18],
  [12, 11],
  [12, 13],
  [13, 12],
  [13, 14],
  [14, 13],
  [14, 15],
  [15, 14],
  [15, 19],
  [16, 3],
  [16, 23],
  [17, 7],
  [17, 27],
  [18, 11],
  [18, 31],
  [19, 15],
  [19, 35],
  [20, 21],
  [21, 20],
  [21, 22],
  [21, 36],
  [22, 21],
  [22, 23],
  [23, 16],
  [23, 22],
  [23, 24],
  [24, 23],
  [24, 25],
  [25, 24],
  [25, 26],
  [25, 37],
  [26, 25],
  [26, 27],
  [27, 17],
  [27, 26],
  [27, 28],
  [28, 27],
  [28, 29],
  [29, 28],
  [29, 30],
  [29, 38],
  [30, 29],
  [30, 31],
  [31, 18],
  [31, 30],
  [31, 32],
  [32, 31],
  [32, 33],
  [33, 32],
  [33, 34],
  [33, 39],
  [34, 33],
  [34, 35],
  [35, 19],
  [35, 34],
  [36, 21],
  

## Job Information
Jobs submitted through Superstaq contain the following information:

* `job_id()`: Unique identifier for the job.
* `status()`: Overall status of the job (Submitted, Queued, Running, Failed, Canceled, or Done). Note that an `index` argument can be passed optionally to retreive the status of a specific job indexed by `index` in a batch job. 
* `target()`: Device the job was run on.
* `counts()`: Counts from the result of the job run. Note this can return a list, so you must specify an `index` if you want to retrieve a specific counts dictionary (corresponding to the circuit with the same `index`).
* `num_qubits()`: Number of qubits for the job. Note this can return a list, so you must specify an `index` if you want to retrieve the corresponding number of qubits to a particular circuit. 
* `repetitions()`: Number of repetitions for the job.
* `input_circuits()`: Retrieves original (i.e., not compiled) circuit(s) for job. Note this returns a list and you must specify an `index` if you want to retrieve a single/specific circuit.
* `compiled_circuits()`: Retrieves compiled circuit(s) from submitted job. Note this returns a list and you must specify an `index` if you want to retrieve a single/specific circuit.

Note that jobs live in our database for a limited amount of time. Typically, they have a lifespan of 1 year.

In [6]:
# Creating a circuit using Cirq
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.measure(qubits[0]),
    cirq.measure(qubits[1]),
)

# Submitting the circuit to IBM's Kingston QPU
job = service.create_job(
    circuit, repetitions=100, method="dry-run", target="ibmq_kingston_qpu"
)  # Specify "dry-run" as the method to submit & run a Superstaq simulation

In [7]:
job.job_id()

'6ed89542-1dc5-4b39-a314-175a46fe87cc'

In [8]:
job.status()

'Done'

In [9]:
job.target()

'ibmq_kingston_qpu'

In [10]:
job.counts(index=0)

{'11': 42, '00': 58}

In [11]:
job.num_qubits(index=0)

2

In [12]:
job.repetitions()

100

In [13]:
job.input_circuits(index=0)

0: ───H───@───M───
          │
1: ───────X───M───

In [14]:
job.compiled_circuits(index=0)

89: ───Rz(0.5π)───X^0.5───────────@───────────│───M('q(0)')───
                                  │           │
90: ───Rz(0.5π)───X^0.5───Rz(π)───@───X^0.5───│───M('q(1)')───

Additionally, you may cancel the current job (or, optionally, a sub-job by passing an `index` argument) if it is not in a terminal state like so:

In [15]:
job.cancel()

/home/rich/software/super.tech/work/client/cirq-superstaq/cirq_superstaq/job.py:197: SuperstaqWarning: Job '6ed89542-1dc5-4b39-a314-175a46fe87cc' cannot be cancelled: it already has status Done
  self._client.cancel_jobs(ids_to_cancel, **kwargs)


Lastly, you may also retrieve the information described above on a previously submitted `cirq-superstaq` job with `service.get_job('<job_id>')`:

In [16]:
job_id = job.job_id()  # Here we use the job ID from above, but this can be any old job ID
job_old = service.get_job(job_id)
job_old.input_circuits(index=0)

0: ───H───@───M───
          │
1: ───────X───M───